In [6]:
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import mnist
from keras.utils import to_categorical

In [7]:
valid_rate=0.3
batchsize=100

In [8]:
from sklearn.model_selection import train_test_split
(x_train_valid, y_train_valid), (x_test, y_test) = mnist.load_data()
x_train_valid=x_train_valid.reshape(-1, 28,28,1)
X_test=x_test.reshape(-1, 28,28,1)
y_train_valid = to_categorical(y_train_valid, 10)
y_test = to_categorical(y_test, 10)
X_train, X_valid, y_train, y_valid = train_test_split(x_train_valid, y_train_valid,test_size=valid_rate)

In [9]:
train_generator=ImageDataGenerator(rescale=1./255,horizontal_flip=True)
test_generator=ImageDataGenerator(rescale=1./255)

In [10]:
g_train=train_generator.flow(X_train,y_train,batch_size=batchsize)
g_valid=test_generator.flow(X_valid,y_valid)
g_test=test_generator.flow(X_test,y_test)

In [11]:
from keras.layers import Input,Conv2D,Dropout,MaxPooling2D,Flatten,Dense
from keras.models import Model

def get_cnn():
    inputs = Input((28,28,1))
    kernel_size = (3, 3)
    max_pool_size = (2, 2)
    cnn = Conv2D(64, kernel_size, padding='same', activation='relu')(inputs)
    cnn = Dropout(0.1)(cnn)
    cnn = Conv2D(64, kernel_size, padding='same', activation='relu')(cnn)
    cnn = Dropout(0.1)(cnn)
    cnn = Conv2D(64, kernel_size, padding='same', activation='relu')(cnn)
    cnn = Dropout(0.1)(cnn)
    cnn = MaxPooling2D(pool_size=max_pool_size, strides=(2, 2))(cnn)

    cnn = Conv2D(64, kernel_size, padding='same', activation='relu')(cnn)
    cnn = Dropout(0.1)(cnn)
    cnn = Conv2D(64, kernel_size, padding='same', activation='relu')(cnn)
    cnn = Dropout(0.1)(cnn)
    cnn = Conv2D(64, kernel_size, padding='same', activation='relu')(cnn)
    cnn = MaxPooling2D(pool_size=max_pool_size, strides=(2, 2))(cnn)

    fc = Flatten()(cnn)
    fc = Dense(1024, activation='relu')(fc)
    predictions = Dense(10, activation='softmax')(fc)
    model = Model(inputs=inputs, outputs=predictions)

    model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
    return model

In [ ]:
model=get_cnn()
history=model.fit_generator(g_train,epochs=3,steps_per_epoch=len(g_train),validation_data=g_valid,validation_steps=len(g_valid))

Epoch 1/3


In [ ]:
model.predict_generator(g_test)